# Week2_1 Assignment

# [BASIC](#Basic)
- BERT 모델의 hidden state에서 **특정 단어의 embedding을 여러 방식으로 추출 및 생성**할 수 있다.

# [CHALLENGE](#Challenge)
- **cosine similarity 함수를 구현**할 수 있다. 
- **단어들의 유사도**를 cosine similarity로 비교할 수 있다. 

# [ADVANCED](#Advanced)
- 문장 embedding을 구해 **문장 간 유사도**를 구할 수 있다.

### Reference
- [BERT word embedding & sentence embedding tutorial 영문 블로그](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/#33-creating-word-and-sentence-vectors-from-hidden-states)

In [1]:
import os
import sys
import pandas as pd
import numpy as np 
import torch
import random

## Basic

### BERT 모델과 토크나이저 로드   
- 두 사람의 대화에서 (단어 및 문장의) embedding을 생성하고자 한다. 아래 대화를 BERT 모델에 입력해 출력값 중 "hidden states"값을 가져오자.
- `Hidden States`는 3차원 텐서를 가지고 있는 list 타입이다. List에는 BERT 모델의 각 layer마다의 hidden state 3차원 텐서를 갖고 있으며 각 텐서는 (batch_size, sequence_length, hidden_size) shape을 가진다. BERT-base 모델은 12 layer를 갖고 있고 이와 별도로 Embedding Layer 1개를 더 갖고 있기 때문에 `len(hidden states)`는 13개가 된다. 
    - batch_size: 학습 시 설정한 배치 사이즈. 또는 BERT 모델에 입력된 문장의 개수
    - sequence_length: 문장의 token의 개수. 
    - hidden size: token의 embedding size 
- Reference
    - [BertTokenizer.tokenize() 함수의 매개변수 설명](https://huggingface.co/transformers/v3.0.2/main_classes/tokenizer.html#transformers.PreTrainedTokenizer.__call__)
    - [BERTModel.forward() 함수의 매개변수 및 리턴 값 설명](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel.forward)

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 9.9 MB/s 
     |████████████████████████████████| 596 kB 62.1 MB/s 
     |████████████████████████████████| 895 kB 57.7 MB/s 
     |████████████████████████████████| 6.5 MB 56.6 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import BertTokenizer, BertModel

In [4]:
tokenizer_bert = BertTokenizer.from_pretrained("bert-base-cased")
model_bert = BertModel.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
normal_person = ["what do you do when you have free time?"]
nerd = ["I code. code frees my minds, body and soul."]
normal_person.append("(what a nerd...) coding?")
nerd.append("Yes. coding is the best thing to do in the free time.")

for i in range(len(normal_person)):
    print(f"Normal Person asked: {normal_person[i]}")
    print(f"Nerd answers: {nerd[i]}")

Normal Person asked: what do you do when you have free time?
Nerd answers: I code. code frees my minds, body and soul.
Normal Person asked: (what a nerd...) coding?
Nerd answers: Yes. coding is the best thing to do in the free time.


In [6]:
# 매개변수 설명
# truncation <- max_len 넘어가지 않도록 자르기
# padding <- max(seq_len, max_len) zero padding
# return_tensors <- return 2d pytorch tensor 

inputs = tokenizer_bert(
    text = normal_person,
    text_pair = nerd,
    truncation = True,
    padding = "longest", 
    return_tensors='pt'
    )

print(inputs['input_ids'].shape)

torch.Size([2, 28])


In [7]:
inputs
# input_ids : 토크나이저의 토큰에 대한 token ID
## [PAD] : 두 sequence의 토큰 개수가 맞지 않아, 토큰이 적은 쪽에 적은만큼 [PAD] 토큰을 붙여 줌 (padding)
# token_type_ids : 몇번째 문장에 속하는가 문장을 구분하는 segment id
# attention_mask : 시퀀스에 어떤 요소가 토큰이고, 패딩요소인지 나타내는 mask id

{'input_ids': tensor([[  101,  1184,  1202,  1128,  1202,  1165,  1128,  1138,  1714,  1159,
           136,   102,   146,  3463,   119,  3463,  1714,  1116,  1139, 10089,
           117,  1404,  1105,  3960,   119,   102,     0,     0],
        [  101,   113,  1184,   170, 24928,  2956,   119,   119,   119,   114,
         19350,   136,   102,  2160,   119, 19350,  1110,  1103,  1436,  1645,
          1106,  1202,  1107,  1103,  1714,  1159,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])}

In [8]:
# decoding
for i in range(len(inputs['input_ids'])): # len(inputs['input_ids']) : 총 문장(sequence)의 갯수
    print(f"Coversation {i} -> '{tokenizer_bert.decode(inputs['input_ids'][i])}'")

Coversation 0 -> '[CLS] what do you do when you have free time? [SEP] I code. code frees my minds, body and soul. [SEP] [PAD] [PAD]'
Coversation 1 -> '[CLS] ( what a nerd... ) coding? [SEP] Yes. coding is the best thing to do in the free time. [SEP]'


In [9]:
# "code" 단어의 token id(각 단어에게 고유하게 주어진 id)를 출력
tokenizer_bert.encode('code', add_special_tokens=False)

[3463]

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


In [11]:
# 입력 데이터와 BERT 모델을 "GPU" 장치로 로드함
inputs = inputs.to(device)
model_bert.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [12]:
# 입력 데이터를 BERT 모델에 넣어 출력값을 가져옴
outputs = model_bert(
    **inputs, 
    output_hidden_states=True
    )

In [13]:
outputs.keys()

# last_hidden_state : 모델의 마지막 hidden_layer의 출력값들
# pooler_output : 마지막 층 모델의 출력값 중 가장 첫번째 [CLS] 토큰의 벡터
# hidden_states : 전체 hidden_layer들(BERT는 1개의 embedding layer + BERT 모델의 12개의 hidden layer = 총 13개 layers를 갖고 있다.)의 출력값들

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])

In [14]:
hidden_states = outputs['hidden_states']
print(f"# layers : {len(hidden_states)}")
print(f"tensor shape in each layer : {hidden_states[-1].shape}") # torch.Size([input sequence 갯수, 28개의 토큰들, 각 토큰들의 임베딩 사이즈(BERT에서 자체적으로 768로 지정)])

# layers : 13
tensor shape in each layer : torch.Size([2, 28, 768])


###  Q1. 1번째 sequence (문장)에서 "code"라는 단어의 인덱스를 모두 반환하라.
- "code" 단어는 총 2개 존재 

In [15]:
def get_index(seq, word):
  token_id = tokenizer_bert.encode(word, add_special_tokens=False)
  index = (seq == token_id[0]).nonzero() 
  #'code'에 대한 token_id : [3463] → 3463만 빼오기 위해 token_id[0] 사용
  # 그래야 seq(=seq의 token_ids 담아놓은 리스트)의 token_id들 중에서 'code'의 token_id인 3463와 일치하는 곳은 True, 일치하지 않는 곳은 False로 출력가능
  # 이 출력을 바탕으로 .nonzero()를 사용해 True(=1)인 곳의 index만 가져올 수 있다.
  return index

# input
# seq1: 1번째 sequence
# token: 단어
seq1 = inputs['input_ids'][0] # 두 seq 중 첫번째 seq
token = "code"

# output
token_index = get_index(seq1, token)
print(token_index)

tensor([[13],
        [15]], device='cuda:0')


### Q2. 1번째 sequence의 1번째 "code" 토큰의 embedding을 여러가지 방식으로 구하고자 한다. BERT hidden state를 다음의 방식으로 인덱싱해 embedding을 구하라
- 1 layer
- last layer
- sum all 12 layers
- sum last 4 layers
- concat last 4 layers
- average last 4 layers

In [16]:
print("layer의 수: ",len(hidden_states))
print("한 layer에 들어가는 sequence의 수: ",len(hidden_states[-1]))
print("한 layer의 한 sequence의 token의 수: ",len(hidden_states[-1][0]))
print("한 layer의 한 sequence의 한 token의 embedding_size: ", len(hidden_states[-1][0][0]))

layer의 수:  13
한 layer에 들어가는 sequence의 수:  2
한 layer의 한 sequence의 token의 수:  28
한 layer의 한 sequence의 한 token의 embedding_size:  768


In [35]:
# 1. combine BERT layers (to make this one whole big tensor)
token_embeddings = torch.stack(hidden_states, dim=0)

# torch.Size([총 layer 개수(embedding layer 포함), sequence 개수, token 개수, 토큰 당 embedding size(=features) ])
token_embeddings.shape

torch.Size([13, 2, 28, 768])

In [ ]:
# 2. creating word and sentence vectors from hidden states

## token들이 갖고 있는 각각의 unique한 vector 가지고 싶지만, 지금 단계에서는 각 13개의 layer에서 token은 13개의 768길이의 token vector을 가지고 있다.
## token들의 단일화된 개인 vector를 갖기 위해서는 각 레이어에 있는 vector들을 combine 해야한다.
## 여기서 고민은, 어떤 레이어에서 나온 vector들을 combine해야지 해당 token을 더 잘 표현하는 vector일 것인가이다.
## 그래서 밑에서 여러방법으로 layer들의 vector들을 combine해보는 것.

In [61]:
# bert의 전체 12 layer들의 첫번재 문장의 첫번째 code에 대한 embeddings(첫 embedding_layer 제외)

layers12_sequence1_code1_token_embeddings = token_embeddings[1:, 0, 13, :]

# torch.Size([12개 레이어, 첫 'code'에 대한 embedding_size(=feature)])
layers12_sequence1_code1_token_embeddings.shape

torch.Size([12, 768])

In [63]:
# 12 layers의 각각의 embedding들을 합쳐버림!
sum = torch.sum(layers12_sequence1_code1_token_embeddings, dim=0)
sum.shape

torch.Size([768])

___

In [105]:
# 1 layer (hidden_state[0]은 BERT의 첫 layer가 아니라, embedding layer)
first_layer_emb = hidden_states[1][0][13] # bert의 12 layers 중 첫 sequence의 첫 'code'의 embedding vector => torch.Size([해당 embedding vector의 size])
print(first_layer_emb.shape)

# last layer 
last_layer_emb = hidden_states[-1][0][13]
print(last_layer_emb.shape)

# sum all 12 layers
sum_all_layer_emb = torch.sum(token_embeddings[1:, 0, 13, :], dim=0)
print(sum_all_layer_emb.shape)

# sum last 4 layers
sum_last4_layer_emb = torch.sum(token_embeddings[-4:, 0, 13, :], dim=0)
print(sum_last4_layer_emb.shape)

# concat last 4 layers
concat_last4_layer_emb = torch.cat((token_embeddings[:, 0, 13, :][-1], token_embeddings[:, 0, 13, :][-2], token_embeddings[:, 0, 13, :][-3], token_embeddings[:, 0, 13, :][-4]), dim=0)
print(concat_last4_layer_emb.shape)

# mean last 4 layers
mean_last4_layer_emb = torch.mean(token_embeddings[-4:, 0, 13, :], dim=0)
print(mean_last4_layer_emb.shape)

torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([3072])
torch.Size([768])


## Challenge

### Q3. `sum_last_four_layer` 방식으로 1번째 sequence의 2개의 "code" 토큰 사이의 코사인 유사도를 계산하라

In [124]:
def cosine_similarity_manual(x, y, small_number=1e-8):
  x_np = x.detach().cpu().numpy()
  y_np = y.detach().cpu().numpy()
  dot_product = np.dot(x_np,y_np)
  l2_norm = (np.sqrt(np.sum(np.square(x_np)))* np.sqrt(np.sum(np.square(y_np))))
  result = dot_product / l2_norm

  return result

# input
# x: 1번째 sequence의 1번째 "code"의 sum_last_four_layer 방식 embedding => idx: 13
# y: 1번째 sequence의 2번째 "code"의 sum_last_four_layer 방식 embedding => idx: 15
x = torch.sum(token_embeddings[-4:, 0, 13, :], dim=0)
y = torch.sum(token_embeddings[-4:, 0, 15, :], dim=0)

# output
score = cosine_similarity_manual(x, y)
print(score)

0.84265727


### Q4. 2번째 sequence에서 "coding"이라는 토큰의 위치를 반환하라

In [125]:
# Q1과 동일한 문제 

# input
# seq1: 2번째 sequence
# token: 단어
seq2 = inputs['input_ids'][1]
token = "coding"

# output
# Q1에서 구현한 함수 사용
token_index = get_index(seq2, token)
print(token_index)

tensor([[10],
        [15]], device='cuda:0')


### Q5. `concat_last4_layer_emb` 방식으로 2번째 sequence의 2개의 "coding" 토큰 사이의 코사인 유사도를 계산하라

In [126]:
# Q3과 동일한 문제

# input
# x: 2번째 sequence의 1번째 "coding"의 concat_last4_layer_emb => idx : 10
# y: 2번째 sequence의 2번째 "coding"의 concat_last4_layer_emb => idx : 15
x = torch.cat((token_embeddings[:, 1, 10, :][-1], token_embeddings[:, 1, 10, :][-2], token_embeddings[:, 1, 10, :][-3], token_embeddings[:, 1, 10, :][-4]), dim=0)
y = torch.cat((token_embeddings[:, 1, 15, :][-1], token_embeddings[:, 1, 15, :][-2], token_embeddings[:, 1, 15, :][-3], token_embeddings[:, 1, 15, :][-4]), dim=0)

# output
# Q3에서 구현한 함수 사용
score = cosine_similarity_manual(x, y)
print(score)

0.8681785


### Q6. 2번째 sequence에서 랜덤하게 토큰 하나를 뽑아보자. 그 랜덤 토큰과 2번째 sequence의 2번째 "coding" 토큰의 코사인 유사도를 계산해보자

In [176]:
# input
# random_idx: random 모듈 사용하여 뽑은 2번째 sequence에서의 랜덤 토큰의 인덱스
# random_word: random_idx에 해당하는 단어
# x: 2번째 sequence의 2번째 "coding" 토큰의 concat_last4_layer_emb
# y: 랜덤 토큰의 concat_last4_layer_emb

seq2 = inputs['input_ids'][1]
random_idx = random.choice(range(len(seq2)))
random_word = tokenizer_bert.decode(seq2[random_idx])

x = torch.cat((token_embeddings[:, 1, 15, :][-1], token_embeddings[:, 1, 15, :][-2], token_embeddings[:, 1, 15, :][-3], token_embeddings[:, 1, 15, :][-4]), dim=0)
y = torch.cat((token_embeddings[:, 1, random_idx, :][-1], token_embeddings[:, 1, random_idx, :][-2], token_embeddings[:, 1, random_idx, :][-3], token_embeddings[:, 1, random_idx, :][-4]), dim=0)

# output
# Q3에서 구현한 함수 사용
score = cosine_similarity_manual(x, y)
print(score)

0.65392417


In [177]:
random_word

'i s'

## Advanced

### Q7. 1번째 sequence와 2번째 sequence의 문장 유사도를 구해보자. 문장의 엠베딩은 마지막 레이어의 첫번째 토큰 ('[CLS]')으로 생성한다.

In [132]:
# input
# x: 1번째 sequence의 embedding
# y: 2번째 sequence의 embedding
x = hidden_states[-1][0][0]
y = hidden_states[-1][1][0]

# output
# Q3에서 구현한 함수 사용
score =  cosine_similarity_manual(x, y)
print(score)

0.8130239
